In [17]:
import gzip
import numpy as np

def open_images(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1, 28, 28)\
            .astype(np.float32)


def open_labels(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=8)

X_train = open_images("data/fashion/train-images-idx3-ubyte.gz")
y_train = open_labels("data/fashion/train-labels-idx1-ubyte.gz")

#### to_categorical (one-hot-encoding)
Nimmt alle möglichen Werte in einem Array (z.B. bei uns Zahlen von 0-9, die die unterschiedlichen Kleidungsstück-Kategorien repräsentieren und wandelt jeden Wert in ein Array um, das nur an der Stelle eine 1 hat, an der die Kategorie übereinstimmt. Alle anderen Werte sind 0.

z.B.

[2,0,1] -> [[0,0,1],[1,0,0],0,1,0]


In [18]:
from keras.utils import to_categorical

# to_categorical: 
y_train_categorical = to_categorical(y_train)
X_train_mapped = X_train.reshape(60000, 784)


In [19]:
# Sequential: bedeutet, dass man Ebene für Ebene ein Modell definieren kann
from keras.models import Sequential
# Dense: Standard-Layer. Bedeutet, dass alle möglichen Pfeile zwischen allen Neuronen erstellt werden (alles mit allem verknüpfen)
from keras.layers import Dense

model = Sequential()

# Hidden layer 
#  100 Neuronen
#  input_shape: Eingabeformat der Daten (bsw. 784 Zahlen für das 28*28px Image)
#  (784,): Tuple - Python braucht für ein Tuple mit nur einem Wert ein ,
model.add(Dense(100, activation="sigmoid", input_shape=(784,)))

# Ausgabe-Layer hat jetzt 10 Ausgänge für unsere 10 Kategorien 
model.add(Dense(10, activation="sigmoid"))

# generiert effizienteren Code. Nötig, falls es später auf einer GPU ausgeführt wird
# optimizer: Welches Verfahren soll verwendet werden, um die Gewichte zu verstellen (sdg: stochastic gradient descent)
# loss: Output-Format & ist eine Aussage über die Größe des Fehlers, der beim Vergleich von x und was als y herauskommen sollte, entsteht. Die Kosten, die man versucht zu minimieren -> (y - ^y) im Quadrat (Fehlergröße)
#           - binary_crossentropy: wenn man wie vorher ja/nein bestimmen möchte
#           - categorical_crossentropy: wenn man Kateorien bestimmen möchte
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(
    X_train_mapped,
    y_train_categorical,
    epochs =10,
    batch_size=1000
)


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3231 - loss: 2.0133
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6624 - loss: 1.3407
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7036 - loss: 1.1313
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7341 - loss: 1.0058
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7497 - loss: 0.9184
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7592 - loss: 0.8582
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7749 - loss: 0.8034
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7801 - loss: 0.7676
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7855 - loss: 0.7376
Epoch 10/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7925 - loss: 0.7078


#### Trainingserfolg prüfen

In [24]:
X_test = open_images("data/fashion/t10k-images-idx3-ubyte.gz").reshape(-1, 784)
y_test = to_categorical(open_labels("data/fashion/t10k-labels-idx1-ubyte.gz"))
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - accuracy: 0.7798 - loss: 0.7159


[0.7216154336929321, 0.7778000235557556]

#### Einzelne Werte testen

In [35]:
# Voraussage ermitteln
pred = model.predict(X_test)

# argmax: gibt die Stelle des größten Wertes im Array zurück (jeweils bei der Voraussage und dem eigentlichen Testwert)
print(np.argmax(pred[0]) == np.argmax(y_test[0]))


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step
True
